# Monometallic Nanocluster Design

In this module, we introduce the **MatOpt** interface for representing material properties and specifying optimization problems. 

We have designed the interface with severl goals in mind:

1. To **simplify the representation of nanostructured materials,** streamlining the creation of materials optimization problems. 
2. To provide a simple interface so that users **do not need to understand the details of building mathematical optmization models** or the syntax of the Pyomo package. 
3. To **automate many of the common steps of materials optimization,** speeding up the development of new models. 

As an example system, we will consider the minimization of cohesive energy in nanoclusters, recently demonstrated in:

Isenberg, N. M., et al., "Identification of Optimally Stable Nanocluster Geometries via Mathematical Optimization and Density Functional Theory," *Molecular Systems Design & Engineering*, 2019. DOI: [10.1039/C9ME00108E](https://doi.org/10.1039/C9ME00108E). 

We seek to identify the geometry that minimizes the cohesive energy of a nanocluster on the face-centered cubic (FCC) lattice. 
As a model for cohesive energy, we use model based on the square-root of coordination number, refered to as the Tomanek model [[1]](https://doi.org/10.1103/PhysRevB.28.665).
In the equation below, we define the normalized cohesive energy, as the normalized contribution of the square root of the coordination number. 

$$\hat{E}^{\text{surf}} = \frac{1}{N \sqrt{12}} \displaystyle \sum_i \sqrt{CN_i} $$

In the following sections, we demonstrate the basic approach for importing the MatOpt package, specifying the design space, formulating an optimization model, solving the optimization problem, and then outputting results. 

## Importing Packages

We start by importing several standard Python modules for convenience. 

In [4]:
import numpy as np
from math import sqrt

Next, we import MatOpt.

In [5]:
from idaes.apps.matopt import *

## Representing Materials
To begin formulating a material optimization problem, we need several pieces of information about the design space.
Our goal is to generate a data structure for representing the choices in the design space, namely the choice of where to place atoms on FCC lattice sites. 

First, we define an ***FCCLattice*** object that holds the information about what sites should be included and which sites should be considered neighbors. 
As argument to the lattice object, we are required to provide the interatomic distance. 

In [6]:
Lat = FCCLattice(IAD=2.770)

Next, we define a ***Canvas*** object that links Cartesian coordinates to more abstract graph consisting of sites and neighbors.
We incrimentally construct a Canvas by first introducing a site at the origin of the coordinate system. 
Then, we add "two shells" of neighbors, meaning that we introduce a shell of sites neighboring to the origin (12 for the FCC lattice) and then introduce another shell of neighbors to that group (42 additional sites, for a total of 55 sites). 
The lattice object provides a *getNeighbors* method to identify these neighbors.

In [7]:
Canv = Canvas()
Canv.addLocation(np.array([0,0,0],dtype=float))
Canv.addShells(2,Lat.getNeighbors)

Finally, we define a list of ***Atom*** objects to represent the building blocks of our materials. 
We then use a ***Design*** object to represent the conjunction of a Canvas with a specific arrangement of building blocks.
The Design object will be used to represent the material decisions made during the solution of material optimization models. 

Before applying optimization, we can use the Design object to plot the sites of the Canvas and ensure that we constructed the intended design space. 
We include several parsers to basic crystal structure file formats such as [XYZ](https://openbabel.org/docs/dev/FileFormats/XYZ_cartesian_coordinates_format.html), [PDB](https://www.rcsb.org/pdb/static.do?p=file_formats/pdb/index.html), [POSCAR](https://cms.mpi.univie.ac.at/vasp/guide/node59.html), and [CFG](http://li.mit.edu/Archive/Graphics/A/index.html#standard_CFG). 

In [8]:
Atoms = [Atom('Pt')]
D = Design(Canv,Atom('Pt'))
D.toPDB('canvas_sites.pdb')

## Building a Model

To hold the materials information, we create a ***MatOptModel*** object. 
This will hold information about the relevant Canvas, Atoms, and material conformations that may be present in a system. 
Additionally, we define a parameter for the desired size of the cluster which will be utilized later by several methods. 

In [9]:
N = 22
m = MatOptModel(Canv,Atoms)

The MatOptModel additionally hold lists of  ***MaterialDescriptor*** objects that define the relevant material desriptors. 
By default, several universal site descriptors are pre-defined in the model. 
From these, all other material descriptors can be defined.

| Descriptor    | Explanation |
|:--------------|:------------|
| ***m.Yik***   | Presence of a building block of type k at site i |
| ***m.Yi***    | Presence of any type of building block at site i |
| ***m.Xijkl*** | Presence of a building block of type k at site i and a building block of type l at site j |
| ***m.Xij***   | Presence of any building block at site i and any building block at site j |
| ***m.Cikl***  | Count of neighbors of type l next to a building block of type k at site i |
| ***m.Ci***    | Count of any type of neighbors next to a building block at site i |

User-specified descriptors are defined by ***DescriptorRule*** objects in conjunction with ***Expr*** expression objects. 
Available expressions include:

| Expression               | Explanation |
|:-------------------------|:------------|
| ***LinearExpr***         | Multiplication and addition of coefficients to distinct MaterialDescriptors |
| ***SiteCombination***    | Summation of site contributions from two sites |
| ***SumNeighborSites***   | Summation of site contributions from all neighboring sites |
| ***SumNeighborBonds***   | Summation of bond contributions to all neighboring sites |
| ***SumSites***           | Summation across sites |
| ***SumBonds***           | Summation across bonds |
| ***SumSiteTypes***       | Summation across site types |
| ***SumBondTypes***       | Summation across bond types |
| ***SumSitesAndTypes***   | Summation across sites and site types |
| ***SumBondsAndTypes***   | Summation across bonds and bond types |
| ***SumConfs***           | Summation across conformation types |
| ***SumSitesAndConfs***   | Summation across sites and conformation types |

Several types of DescriptorRules are available. 

| Rule                          | Explanation |
|:------------------------------|:------------|
| ***LessThan***                | Descriptor less than or equal to an expression |
| ***EqualTo***                 | Descriptor equal to an expression |
| ***GreaterThan***             | Descriptor greater than or equal to an expression |
| ***FixedTo***                 | Descriptor fixed to a scalar value |
| ***PiecewiseLinear***         | Descriptor equal to the evaluation of a piecewise linear function |
| ***Implies***                 | Indicator descriptor that imposes other constraints if equal to 1 |
| ***NegImplies***              | Indicator descriptor that imposes other constraints if equal to 0 |
| ***ImpliesSiteCombination***  | Indicator bond-indexed descriptor that imposes constraints on the two sites |
| ***ImpliesNeighbors***        | Indicator site-indexed descriptor that imposes constraints on neighboring sites |

From the combination of pre-defined descriptors, expressions, and rules we can specify a wide variety of other descriptors. 

In the context of nanocluster cohesive energy minimization, we would first like to define the square root of the coordination number. 
We achieve this by calling the ***addSitesDescriptor*** method on MatOptModel, passing the information necessary to create a ***PiecewiseLinear*** rule to correctly define the square root of coordination at the integer coordination number values. 
Note that we use ***m.Ci***, the pre-defined basic variable for the count of neighboring building blocks and equivalent to the coordination number in this system, as the argument for the piecewise linear function. 
We use basic Python lists to express the data for the piecewise linear function values at integer numbers of coordination. 

In [10]:
Vals = [sqrt(CN) for CN in range(0,13)]
BPs = [CN for CN in range(0,13)]
m.addSitesDescriptor('CNRi',bounds=(0,sqrt(12)),integer=False,
                     rules=PiecewiseLinear(values=Vals,
                                           breakpoints=BPs,
                                           input_desc=m.Ci))

Next, we define a global (i.e., not indexed by sites or bonds) descriptor for the cohesive energy of the nanocluster. 
We us a simple ***EqualTo*** rule to set this descriptor equal to a normalized sum of contributions from the square root coordination number descriptor. 

In [11]:
m.addGlobalDescriptor('Ecoh',rules=EqualTo(SumSites(desc=m.CNRi,
                                                    coefs=(1/(N*sqrt(12))))))

Finally, we create a descriptor for the size of the nanocluster. 
We set bounds on this descriptor to effectively constrain the design space to only include clusters of the desired size, *N*. 

In [12]:
m.addGlobalDescriptor('Size',bounds=(N,N),
                      rules=EqualTo(SumSites(desc=m.Yi)))

## Solving the Model
Once all the relevant information in the model is provided, we search for optimal designs that maximize one of the descriptors. 
In this example, we provide the descriptor for coehisver energy as the target functionality. 
Additionally, we specify a time limit in seconds as a keyword argument to the maximize method. 
For more information, see the documentation of the maximize function, available in the source code or by using the Python *help* function.

In [13]:
help(MatOptModel.maximize)
help(MatOptModel.optimize)

In [14]:
D = None
try:
    D = m.maximize(m.Ecoh,tilim=100)
except:
    print('MaOpt can not find usable solver (CPLEX or NEOS-CPLEX)')

## Processing Results

If a result is found, we can write it to file and plot with visualization software. 
We provide interfaces to several standard crystal structure file formats, including [XYZ](https://openbabel.org/docs/dev/FileFormats/XYZ_cartesian_coordinates_format.html), [PDB](https://www.rcsb.org/pdb/static.do?p=file_formats/pdb/index.html), [POSCAR](https://cms.mpi.univie.ac.at/vasp/guide/node59.html), and [CFG](http://li.mit.edu/Archive/Graphics/A/index.html#standard_CFG). 

In [15]:
if(D is not None):
    D.toPDB('result.pdb')